## Prepare Library

In [2]:
import os
import torch 
import pytorch_lightning as pl
import torchmetrics
from torchmetrics.classification.accuracy import Accuracy
import pandas as pd 
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.autograd import Variable 
from tqdm import tqdm
from scipy import integrate
from model import LSTM
from random import sample
import torch.nn as nn
from scipy.signal import find_peaks

## Prepare GPU

In [4]:
print(torch.__version__ )
print(torch.version.cuda)
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

1.11.0+cu113
11.3
True


## Dataset

In [16]:
Index = ['Channel1', 'Channel2', 'Channel3', 'Channel4', 'Channel5', 'Channel6']

Data_path = 'G:/Workspace/Work/TENG-Signal-Classification/dataset/preprocessed'

cases = os.listdir(Data_path)

cases

random.shuffle(cases)

## Load data

In [9]:
test_set = cases[:50]

valid_set = cases[50:100]

train_set = cases[100:]

print('lenght of train set:', len(train_set))
print('lenght of valid set:', len(valid_set))
print('valid set:', valid_set)
print('lenght of test set:', len(test_set))
print('test set:', test_set)

lenght of train set: 200
lenght of valid set: 50
valid set: ['C2_7', 'C2_38', 'C1_3', 'C6_21', 'C1_46', 'C2_19', 'C6_31', 'C5_47', 'C6_30', 'C4_19', 'C5_3', 'C6_9', 'C2_42', 'C3_5', 'C5_30', 'C1_14', 'C1_37', 'C1_8', 'C1_49', 'C2_34', 'C5_23', 'C1_1', 'C2_22', 'C5_17', 'C3_15', 'C5_39', 'C2_47', 'C2_37', 'C1_35', 'C4_21', 'C3_49', 'C4_50', 'C4_2', 'C5_7', 'C3_13', 'C1_50', 'C5_38', 'C4_4', 'C3_25', 'C5_9', 'C4_44', 'C2_5', 'C5_11', 'C4_35', 'C1_43', 'C5_43', 'C2_4', 'C4_13', 'C4_33', 'C6_50']
lenght of test set: 50
test set: ['C5_48', 'C6_33', 'C3_16', 'C1_26', 'C1_34', 'C3_26', 'C5_44', 'C5_28', 'C4_42', 'C6_49', 'C6_14', 'C1_23', 'C6_44', 'C4_6', 'C6_15', 'C4_46', 'C3_39', 'C4_48', 'C6_16', 'C2_46', 'C1_4', 'C4_30', 'C6_40', 'C5_36', 'C3_34', 'C6_46', 'C4_32', 'C1_11', 'C6_2', 'C4_36', 'C5_46', 'C4_3', 'C3_8', 'C3_44', 'C3_33', 'C1_18', 'C2_40', 'C2_11', 'C1_42', 'C6_23', 'C1_25', 'C2_49', 'C6_39', 'C2_31', 'C2_50', 'C6_36', 'C2_30', 'C4_37', 'C4_9', 'C4_29']


In [22]:
x_train_data = [] 
y_train_data = []
x_valid_data = []
y_valid_data = []
x_test_data = []
y_test_data = []

print('| loading train set data..... |')
for case in train_set:
    label = case.split('_')[0]
    #   Read csv
    file = case + '.csv'
    data = pd.read_csv(Data_path + '/' + case + '/' + file, usecols = Index, squeeze = True)
    #   Convert lable into int
    Encode_label = {
            'C1': 0,
            'C2': 1,
            'C3': 2,
            'C4': 3,
            'C5': 4,
            'C6': 5, 
    }
    x_train_data.append(data)
    y_train_data.append(Encode_label[label])
print('| done |')

print('| loading valid set data..... |')
for case in valid_set:
    label = case.split('_')[0]
    #   Read csv
    file = case + '.csv'
    data = pd.read_csv(Data_path + '/' + case + '/' + file, usecols = Index, squeeze = True)
    #   Convert lable into int
    Encode_label = {
            'C1': 0,
            'C2': 1,
            'C3': 2,
            'C4': 3,
            'C5': 4,
            'C6': 5, 
    }
    x_valid_data.append(data)
    y_valid_data.append(Encode_label[label])
print('| done |')

print('| loading test set data..... |')
for case in test_set:
    label = case.split('_')[0]
    #   Read csv
    file = case + '.csv'
    data = pd.read_csv(Data_path + '/' + case + '/' + file, usecols = Index, squeeze = True)
    #   Convert lable into int
    Encode_label = {
            'C1': 0,
            'C2': 1,
            'C3': 2,
            'C4': 3,
            'C5': 4,
            'C6': 5, 
    }
    x_test_data.append(data)
    y_test_data.append(Encode_label[label])
print('| done |')

ss = StandardScaler()

X_train_tensors = []
Y_train_tensors = []
X_valid_tensors = []
Y_valid_tensors = []
X_test_tensors = []
Y_test_tensors = []


print('| train set data to tensor..... |')
## To tensors
for i in range(len(train_set)):
    X = x_train_data[i]
    Y = y_train_data[i]
    X_ss = ss.fit_transform(X)
    
    X_tensors = torch.Tensor(X_ss)
    Y_tensors = torch.tensor(Y)

    X_train_tensors.append(X_tensors.to(device))
    Y_train_tensors.append(Y_tensors.to(device))
print('| done |')

print('| valid data to tensor..... |')
for i in range(len(valid_set)):
    X = x_valid_data[i]
    Y = y_valid_data[i]
    X_ss = ss.fit_transform(X)
    
    X_tensors = torch.Tensor(X_ss)
    Y_tensors = torch.tensor(Y)

    X_valid_tensors.append(X_tensors.to(device))
    Y_valid_tensors.append(Y_tensors.to(device))
print('| done |')

print('| test data to tensor..... |')
for i in range(len(test_set)):
    X = x_test_data[i]
    Y = y_test_data[i]
    X_ss = ss.fit_transform(X)
    
    X_tensors = torch.Tensor(X_ss)
    Y_tensors = torch.tensor(Y)

    X_test_tensors.append(X_tensors.to(device))
    Y_test_tensors.append(Y_tensors.to(device))
print('| done |')

| loading train set data..... |
| done |
| loading valid set data..... |
| done |
| loading test set data..... |
| done |
| train set data to tensor..... |
| done |
| valid data to tensor..... |
| done |
| test data to tensor..... |
| done |


## Training 

In [37]:
num_epochs = 100
learning_rate = 1e-4

n_features = len(Index) #number of features
n_hidden = 256 #number of features in hidden state
n_layers = 2 #number of stacked lstm layers
n_classes = 6 #number of output classes 
dropout = 0.5
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [38]:
lstm = LSTM(n_features, n_classes, n_hidden, n_layers, dropout, device) #our lstm class
lstm.to(device)
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate) 
accuracy = Accuracy(task="multiclass", num_classes=6)
train_loss = []
train_acc = []

trainer_path = 'G:/Workspace/Work/TENG-Signal-Classification/trainer'

valid_loss = []
valid_acc = []

to_one_hot = [[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]

for epoch in tqdm(range(num_epochs)):
    l = []
    pre = []
    # mix = []
    for index, case in enumerate(train_set):
        # rand_index = random.randint(0,len(train_set) - 1)
        x1 = X_train_tensors[index]
        # x2 = X_train_tensors[rand_index]
        label1 = Y_train_tensors[index]
        # label2 = Y_train_tensors[rand_index]
        one_hot1 = to_one_hot[label1]
        # one_hot2 = to_one_hot[label2]

        # lam = np.random.beta(1, 1)
        # x_mixed_up = lam * x1 + (1 - lam) * x2
        # y_mixed_up = lam * torch.FloatTensor(one_hot1) + (1 - lam) * torch.FloatTensor(one_hot2)

        outputs = lstm.forward(x1) #forward pass
        optimizer.zero_grad() #caluclate the gradient, manually setting to 0

        # obtain the loss function
        step_loss = criterion(torch.unsqueeze(outputs,0), torch.unsqueeze(torch.FloatTensor(one_hot1),0).to(device))
        l.append(step_loss.item())

        outputs = outputs.softmax(dim = 0)
        prediction = torch.argmax(outputs)
        pre.append(prediction)
        step_loss.backward() #calculates the loss of the loss function
        optimizer.step() #improve from loss, i.e backprop

    acc = accuracy(torch.tensor(pre), torch.tensor(Y_train_tensors))
    train_acc.append(acc.item())
    loss = np.mean(l) 
    train_loss.append(loss)
    best_train_loss = min(train_loss)
    
    if epoch < 10:
        print("Epoch: %d | train loss: %1.5f, best: %1.5f | accuracy: %1.5f, best: %1.5f" % (epoch, loss, best_train_loss, acc, max(train_acc))) 
    else:
        print("Epoch: %d| train loss: %1.5f, best: %1.5f | accuracy: %1.5f, best: %1.5f" % (epoch, loss, best_train_loss, acc, max(train_acc))) 

    # if epoch % 5 == 0 or epoch == num_epochs - 1 :
    _l = []
    _pre = []
    for index, case in enumerate(valid_set):

        label = Y_valid_tensors[index]
        one_hot = to_one_hot[label]
        
        outputs = lstm(X_valid_tensors[index]) #forward pass
        step_loss = criterion(torch.unsqueeze(outputs,0), torch.unsqueeze(torch.FloatTensor(one_hot),0).to(device))
        _l.append(step_loss.item())
         
        outputs = outputs.softmax(dim = 0)
        prediction = torch.argmax(outputs)
        
        _pre.append(prediction)
    
    acc = accuracy(torch.tensor(_pre), torch.tensor(Y_valid_tensors)) 
    valid_acc.append(acc.item())
    loss = np.mean(_l)
    if  acc >= 0.8:
        torch.save(lstm.state_dict(), trainer_path + 'epoch_' + str(epoch) +
                   '_loss_' + str(loss) + '_acc_' + str(acc.item() * 100) + '%' + '.pt')
    
    valid_loss.append(loss)
    best_valid_loss = min(valid_loss)
    
    if epoch == num_epochs - 1:
        print("         | valid loss: %1.5f, best: %1.5f | accuracy: %1.5f, best: %1.5f" % (loss, best_valid_loss, acc, max(valid_acc))) 
    else:
         print("         | valid loss: %1.5f, best: %1.5f | accuracy: %1.5f, best: %1.5f" % (loss, best_valid_loss, acc, max(valid_acc))) 


100%|██████████| 100/100 [20:29<00:00, 12.29s/it]


Epoch: 0 | train loss: 1.79635, best: 1.79635 | accuracy: 0.15000, best: 0.15000
         | valid loss: 1.79373, best: 1.79373 | accuracy: 0.20000, best: 0.20000
Epoch: 1 | train loss: 1.79373, best: 1.79373 | accuracy: 0.16000, best: 0.16000
         | valid loss: 1.79478, best: 1.79373 | accuracy: 0.08000, best: 0.20000
Epoch: 2 | train loss: 1.79241, best: 1.79241 | accuracy: 0.17500, best: 0.17500
         | valid loss: 1.79466, best: 1.79373 | accuracy: 0.10000, best: 0.20000
Epoch: 3 | train loss: 1.79081, best: 1.79081 | accuracy: 0.19500, best: 0.19500
         | valid loss: 1.79354, best: 1.79354 | accuracy: 0.10000, best: 0.20000
Epoch: 4 | train loss: 1.79546, best: 1.79081 | accuracy: 0.18000, best: 0.19500
         | valid loss: 1.78273, best: 1.78273 | accuracy: 0.22000, best: 0.22000
Epoch: 5 | train loss: 1.78159, best: 1.78159 | accuracy: 0.23000, best: 0.23000
         | valid loss: 1.77521, best: 1.77521 | accuracy: 0.22000, best: 0.22000
Epoch: 6 | train loss: 1.748